<a href="https://colab.research.google.com/github/GIGIDAGG/PCTO/blob/main/Coco128_%2B_Buoys_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
import torch

In [ ]:
torch.hub.download_url_to_file('https://app.roboflow.com/ds/6fDcQZtyLu?key=NmliLrqyk5', 'roboflow.zip')

In [ ]:
!unzip -q roboflow.zip -d /content/buoys && rm roboflow.zip

In [ ]:
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco128.zip', 'tmp.zip')

In [ ]:
!unzip -q tmp.zip -d /content/datasets && rm tmp.zip

In [ ]:
!cp /content/buoys/train/images/* /content/datasets/coco128/images/train2017/

In [ ]:
!cp /content/buoys/train/labels/* /content/datasets/coco128/labels/train2017/

In [ ]:
import yaml
#with open('/content/buoys/data.yaml', 'r') as file:
#    nuovi_dati = yaml.safe_load(file)
#lista = nuovi_dati['names']
with open('/content/yolov5/data/coco128.yaml', 'r') as file:
    data = yaml.safe_load(file)
#for i in lista:
#  data['names'][len(data['names'])] = i
data['names'][0] = 'buoy'
with open('/content/yolov5/data/coco128.yaml', 'w') as file:
    yaml.dump(data, file)

In [ ]:
%cd /content/yolov5

In [ ]:
%pip install -qr requirements.txt

In [ ]:
!python train.py --img 640 --batch 16 --epochs 60 --data /content/yolov5/data/coco128.yaml --weights yolov5s.pt --cache

In [ ]:
!python detect.py --weights runs/train/exp/weights/last.pt --img 640 --conf 0.25 --source https://youtu.be/gPqqcKuKyaI

In [ ]:
!pip install "wandb==0.12.10"

In [ ]:
#@title inserimento dataset e train su wandb
import wandb
wandb.login(relogin=True)
wandb.init(project='PCTO UNIVPM', entity='gigidagg')
# Crea un nuovo artefatto
artifact = wandb.Artifact('data.yaml', type='dataset')

# Carica i dati delle boe nel tuo artefatto
artifact.add_file('/content/data.yaml', name='data.yaml')

# Aggiungi eventuali metadati o descrizioni
wandb.log({"message": "Dataset delle boe salvato con successo"})

# Salva l'artefatto
wandb.run.log_artifact(artifact)
wandb.run.finish()